Video 58: NLP Tools Part One
========================

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('natural language processing').getOrCreate()
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic, regression, models, are, neat')], ['id', 'sentence'])

In [2]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic, regress...|
+---+--------------------+



In [3]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')
count_tokens = udf(lambda words: len(words), IntegerType())
tokenized = tokenizer.transform(sen_df)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic, regress...|[logistic,, regre...|
+---+--------------------+--------------------+



In [4]:
tokenized.withColumn('token', count_tokens(col('words'))).show()

+---+--------------------+--------------------+-----+
| id|            sentence|               words|token|
+---+--------------------+--------------------+-----+
|  0|Hi I heard about ...|[hi, i, heard, ab...|    5|
|  1|I wish java could...|[i, wish, java, c...|    7|
|  2|Logistic, regress...|[logistic,, regre...|    5|
+---+--------------------+--------------------+-----+



In [5]:
rg_tokenized = regex_tokenizer.transform(sen_df)
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic, regress...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [6]:
from pyspark.ml.feature import StopWordsRemover
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'litle', 'lamb'])], ['id', 'tokens'])
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
remover.transform(sentenceDataFrame).show()

+---+--------------------+-------------------+
| id|              tokens|           filtered|
+---+--------------------+-------------------+
|  0|[I, saw, the, gre...|[saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, litle, lamb]|
+---+--------------------+-------------------+



In [7]:
from pyspark.ml.feature import NGram
wordDataFrame = spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
    (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])], ['id', 'words'])
ngram = NGram(n=2, inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, java, c...|[I wish, wish jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [8]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



Video 59: NLP Tools par Two
========

In [9]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sentenceData = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic regression models are neat')], ['label', 'sentence'])
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I heard about ...|
|    1|I wish java could...|
|    2|Logistic regressi...|
+-----+--------------------+



In [10]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
words_data = tokenizer.transform(sentenceData)
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|
|    1|I wish java could...|[i, wish, java, c...|
|    2|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [11]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(262144,[24417,49...|
|    1|(262144,[20719,24...|
|    2|(262144,[13671,91...|
+-----+--------------------+



In [13]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|1    |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [14]:
from pyspark.ml.feature import CountVectorizer
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' '))
], ['id', 'words'])
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [16]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)
model = cv.fit(df)
result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+



Video 60: Natural Language Processing Code Along Project
========

In [18]:
data = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t')
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [22]:
from pyspark.sql.functions import length
data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [23]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [27]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [29]:
clean_data = clean_data.select('label', 'features')
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [31]:
training, test = clean_data.randomSplit([0.7, 0.3])
spam_detector = nb.fit(training)
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [32]:
test_results = spam_detector.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,3,10,...|[-604.60383357053...|[1.0,5.5835908700...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-1173.6707013026...|[1.0,6.2569128149...|       0.0|
|  0.0|(13459,[0,1,8,16,...|[-659.92738127895...|[1.0,1.6894397593...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-540.46889342806...|[1.0,8.9603271465...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-874.44712253362...|[1.0,9.6523643471...|       0.0|
|  0.0|(13459,[0,1,19,21...|[-839.98736984031...|[1.0,3.1395156583...|       0.0|
|  0.0|(13459,[0,1,21,29...|[-969.45400757933...|[1.0,8.0661844073...|       0.0|
|  0.0|(13459,[0,1,33,46...|[-376.71812937814...|[1.0,2.2024560858...|       0.0|
|  0.0|(13459,[0,1,46,72...|[-616.22709328686...|[0.99744668379002...|       0.0|
|  0.0|(13459,[0

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print('ACC of NB Model:')
print(acc)

ACC of NB Model:
0.9211514497519449
